In [3]:
from catboost import CatBoostClassifier, Pool
from hyperopt import hp, tpe, fmin, Trials
import numpy as np
import pandas as pd
import polars as pl
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.simplefilter('ignore')

In [4]:
train = pl.read_csv('/home/zhenghao/kaggle/train.csv')
test = pl.read_csv('/home/zhenghao/kaggle/test.csv')

# 在测试集中添加 'Response' 列，值为 0，类型为 Int64
test = test.with_columns(pl.lit(0).cast(pl.Int64).alias('Response'))

In [5]:
# 将训练集和测试集合并
df = pl.concat([train, test])

# 将性别列中的'Male'替换为0，'Female'替换为1，并将列类型转换为Int32
df = df.with_columns([
    pl.col('Gender').replace({'Male': 0, 'Female': 1}).cast(pl.Int32),
    # 将Region_Code列类型转换为int
    pl.col('Region_Code').cast(int),
    # 将Vehicle_Age列中的'< 1 Year'替换为0，'1-2 Year'替换为1，'> 2 Years'替换为2，并将列类型转换为Int32
    pl.col('Vehicle_Age').replace({'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}).cast(pl.Int32),
    # 将Vehicle_Damage列中的'No'替换为0，'Yes'替换为1，并将列类型转换为Int32
    pl.col('Vehicle_Damage').replace({'No': 0, 'Yes': 1}).cast(pl.Int32),
    # 将Annual_Premium列类型转换为int
    pl.col('Annual_Premium').cast(int),
    # 将Policy_Sales_Channel列类型转换为int
    pl.col('Policy_Sales_Channel').cast(int)
])

# 将Previously_Insured列和Annual_Premium列合并，并使用factorize函数进行编码，将结果命名为Previously_Insured_Annual_Premium
df = df.with_columns([
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Annual_Premium'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Annual_Premium'),
    # 将Previously_Insured列和Vehicle_Age列合并，并使用factorize函数进行编码，将结果命名为Previously_Insured_Vehicle_Age
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vehicle_Age'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vehicle_Age'),
    # 将Previously_Insured列和Vehicle_Damage列合并，并使用factorize函数进行编码，将结果命名为Previously_Insured_Vehicle_Damage
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vehicle_Damage'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vehicle_Damage'),
    # 将Previously_Insured列和Vintage列合并，并使用factorize函数进行编码，将结果命名为Previously_Insured_Vintage
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vintage'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vintage')
])

# 将df的前train.shape[0]行转换为pandas DataFrame，并赋值给train
train = df[:train.shape[0]].to_pandas()
# 将df的train.shape[0]行之后的所有行转换为pandas DataFrame，并赋值给test
test = df[train.shape[0]:].to_pandas()

# 打印train
train

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Previously_Insured_Annual_Premium,Previously_Insured_Vehicle_Age,Previously_Insured_Vehicle_Damage,Previously_Insured_Vintage
0,0,0,21,1,35,0,1,1,65101,124,187,0,0,0,0,0
1,1,0,43,1,28,0,2,1,58911,26,288,1,1,1,0,1
2,2,1,25,1,14,1,0,0,38043,152,254,0,2,2,1,2
3,3,1,35,1,1,0,1,1,2630,156,76,0,3,0,0,3
4,4,1,36,1,15,1,1,0,31951,152,294,0,4,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,11504793,0,48,1,6,0,1,1,27412,26,218,0,5210,0,0,144
11504794,11504794,1,26,1,36,0,0,1,29509,152,115,1,23274,4,0,176
11504795,11504795,1,29,1,32,1,0,0,2630,152,189,0,18,2,1,456
11504796,11504796,1,51,1,28,0,1,1,48443,26,274,1,14121,0,0,124


In [51]:
'''使用 polars 库读取 CSV 数据。
合并训练集和测试集以便统一预处理。
对类别特征进行编码，如性别、地区代码、车辆年龄、车辆损坏情况等。
创建一些组合特征以增加模型的表达能力。
将预处理后的数据分割回训练集和测试集，并转换为 Pandas 数据框。
Previously_Insured 和 Annual_Premium 的组合

组合方式：将 Previously_Insured 和 Annual_Premium 转换为字符串形式后相加。
编码方式：使用 pd.factorize 对组合后的字符串进行编码。
生成新特征 Previously_Insured_Annual_Premium。
Previously_Insured 和 Vehicle_Age 的组合

组合方式：将 Previously_Insured 和 Vehicle_Age 转换为字符串形式后相加。
编码方式：使用 pd.factorize 对组合后的字符串进行编码。
生成新特征 Previously_Insured_Vehicle_Age。
Previously_Insured 和 Vehicle_Damage 的组合

组合方式：将 Previously_Insured 和 Vehicle_Damage 转换为字符串形式后相加。
编码方式：使用 pd.factorize 对组合后的字符串进行编码。
生成新特征 Previously_Insured_Vehicle_Damage。
Previously_Insured 和 Vintage 的组合

组合方式：将 Previously_Insured 和 Vintage 转换为字符串形式后相加。
编码方式：使用 pd.factorize 对组合后的字符串进行编码。
生成新特征 Previously_Insured_Vintage。
通过这种方式生成的组合特征，能够捕捉到原始特征之间的相互关系，提高模型的表达能力，从而提升模型的预测性能。'''

'使用 polars 库读取 CSV 数据。\n合并训练集和测试集以便统一预处理。\n对类别特征进行编码，如性别、地区代码、车辆年龄、车辆损坏情况等。\n创建一些组合特征以增加模型的表达能力。\n将预处理后的数据分割回训练集和测试集，并转换为 Pandas 数据框。\nPreviously_Insured 和 Annual_Premium 的组合\n\n组合方式：将 Previously_Insured 和 Annual_Premium 转换为字符串形式后相加。\n编码方式：使用 pd.factorize 对组合后的字符串进行编码。\n生成新特征 Previously_Insured_Annual_Premium。\nPreviously_Insured 和 Vehicle_Age 的组合\n\n组合方式：将 Previously_Insured 和 Vehicle_Age 转换为字符串形式后相加。\n编码方式：使用 pd.factorize 对组合后的字符串进行编码。\n生成新特征 Previously_Insured_Vehicle_Age。\nPreviously_Insured 和 Vehicle_Damage 的组合\n\n组合方式：将 Previously_Insured 和 Vehicle_Damage 转换为字符串形式后相加。\n编码方式：使用 pd.factorize 对组合后的字符串进行编码。\n生成新特征 Previously_Insured_Vehicle_Damage。\nPreviously_Insured 和 Vintage 的组合\n\n组合方式：将 Previously_Insured 和 Vintage 转换为字符串形式后相加。\n编码方式：使用 pd.factorize 对组合后的字符串进行编码。\n生成新特征 Previously_Insured_Vintage。\n通过这种方式生成的组合特征，能够捕捉到原始特征之间的相互关系，提高模型的表达能力，从而提升模型的预测性能。'

In [52]:
'''aucs = []
preds = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train['Response'])):
    print(f'### Fold {fold+1} Training ###')

    X_train = train.loc[train_idx, [c for c in train.columns if c not in ['id', 'Response']]]
    y_train = train.loc[train_idx, 'Response']
    X_valid = train.loc[valid_idx, X_train.columns]
    y_valid = train.loc[valid_idx, 'Response']
    X_test = test[X_train.columns]

    X_train_pool = Pool(X_train, y_train, cat_features=X_train.columns.values)
    X_valid_pool = Pool(X_valid, y_valid, cat_features=X_valid.columns.values)
    X_test_pool = Pool(X_test, cat_features=X_test.columns.values)

    model = CatBoostClassifier(
        loss_function='Logloss',
        eval_metric='Logloss',
        learning_rate=0.05,
        iterations=5000,
        depth=9,
        random_strength=0,
        l2_leaf_reg=0.5,
        task_type='GPU',
        devices='1:2:3:4:5:6',
        random_seed=42,
        verbose=False
    )

    model.fit(X=X_train_pool, eval_set=X_valid_pool, verbose=1000, early_stopping_rounds=200)

    pred_valid = model.predict_proba(X_valid_pool)[:, 1]
    preds.append(model.predict_proba(X_test_pool)[:, 1])

    auc = roc_auc_score(y_valid, pred_valid)
    aucs.append(auc)

    print(f'Fold {fold+1} AUC: {auc:.5f}\n')

print(f'\nOverall AUC: {np.mean(aucs):.5f} +/- {np.std(aucs):.5f}')'''

"aucs = []\npreds = []\n\nskf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)\n\nfor fold, (train_idx, valid_idx) in enumerate(skf.split(train, train['Response'])):\n    print(f'### Fold {fold+1} Training ###')\n\n    X_train = train.loc[train_idx, [c for c in train.columns if c not in ['id', 'Response']]]\n    y_train = train.loc[train_idx, 'Response']\n    X_valid = train.loc[valid_idx, X_train.columns]\n    y_valid = train.loc[valid_idx, 'Response']\n    X_test = test[X_train.columns]\n\n    X_train_pool = Pool(X_train, y_train, cat_features=X_train.columns.values)\n    X_valid_pool = Pool(X_valid, y_valid, cat_features=X_valid.columns.values)\n    X_test_pool = Pool(X_test, cat_features=X_test.columns.values)\n\n    model = CatBoostClassifier(\n        loss_function='Logloss',\n        eval_metric='Logloss',\n        learning_rate=0.05,\n        iterations=5000,\n        depth=9,\n        random_strength=0,\n        l2_leaf_reg=0.5,\n        task_type='GPU',\n        d

In [53]:
'''使用 StratifiedKFold 进行5折交叉验证，以确保各折中类别比例相同。
在每一折中，分割训练和验证数据，并创建 CatBoost 数据池。
定义 CatBoostClassifier 模型，并设置参数如损失函数、评估指标、学习率、迭代次数、树深度等。
训练模型，并在验证集上评估模型性能（AUC）。
记录每折的预测结果和AUC值。'''

'使用 StratifiedKFold 进行5折交叉验证，以确保各折中类别比例相同。\n在每一折中，分割训练和验证数据，并创建 CatBoost 数据池。\n定义 CatBoostClassifier 模型，并设置参数如损失函数、评估指标、学习率、迭代次数、树深度等。\n训练模型，并在验证集上评估模型性能（AUC）。\n记录每折的预测结果和AUC值。'

In [6]:
X = train.drop(['id', 'Response'], axis=1)
y = train['Response']
X_test = test.drop(['id', 'Response'], axis=1)

In [9]:
# 定义超参数优化目标函数
def hyperopt_objective(params):
    # 初始化aucs列表
    aucs = []
    # 定义交叉验证
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # 遍历交叉验证的每个折
    for train_idx, valid_idx in skf.split(X, y):
        # 获取训练集和验证集
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        # 定义训练集和验证集的池
        train_pool = Pool(X_train, y_train, cat_features=X_train.columns.values)
        valid_pool = Pool(X_valid, y_valid, cat_features=X_valid.columns.values)

        # 定义CatBoost分类器
        model = CatBoostClassifier(
            loss_function='Logloss',
            eval_metric='Logloss',
            learning_rate=params['learning_rate'],
            iterations=int(params['iterations']),
            depth=int(params['depth']),
            l2_leaf_reg=params['l2_leaf_reg'],
            random_strength=params['random_strength'],
            task_type='GPU',
            devices='0:1:2:3:4:5:6',
            random_seed=42,
            verbose=False
        )

        # 训练模型
        model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=200, verbose=False)
        # 预测验证集
        y_pred = model.predict_proba(valid_pool)[:, 1]
        # 计算auc
        auc = roc_auc_score(y_valid, y_pred)
        # 将auc添加到aucs列表中
        aucs.append(auc)

    # 返回平均auc的负值
    return -np.mean(aucs)

# 定义参数空间
param_space = {
    # 学习率
    'learning_rate': hp.uniform('learning_rate',0.1, 1),
    # 迭代次数
    'iterations': hp.quniform('iterations', 2000, 7000, 100),
    # 树的深度
    'depth': hp.quniform('depth', 5, 13, 1),
    # L2正则化
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 0.1, 5),
    # 随机强度
    'random_strength': hp.uniform('random_strength', 0, 10)
}


In [11]:
# 使用 Hyperopt 进行超参数优化
trials = Trials()
# 使用 fmin 函数进行超参数优化，fn 参数为超参数优化的目标函数，space 参数为超参数空间，algo 参数为优化算法，max_evals 参数为最大迭代次数，trials 参数为试验结果，rstate 参数为随机数种子
best = fmin(
    fn=hyperopt_objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.default_rng(42)
)

# 将最优参数中的 iterations 和 depth 转换为整数类型
best['iterations'] = int(best['iterations'])
best['depth'] = int(best['depth'])
# 打印最优参数
print("Best parameters found: ", best)

  0%|          | 0/50 [00:05<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [ ]:
# 使用最佳参数训练最终模型并预测
train_pool = Pool(X, y, cat_features=X.columns.values)  # 创建训练集
test_pool = Pool(X_test, cat_features=X_test.columns.values)  # 创建测试集

model = CatBoostClassifier(
    loss_function='Logloss',  # 损失函数
    eval_metric='AUC',  # 评估指标
    learning_rate=best['learning_rate'],  # 学习率
    iterations=best['iterations'],  # 迭代次数
    depth=best['depth'],  # 树的深度
    random_strength=best['random_strength'],  # 随机强度
    l2_leaf_reg=best['l2_leaf_reg'],  # L2正则化
    task_type='GPU',  # 任务类型
    devices='0',  # 设备
    random_seed=42,  # 随机种子
    verbose=1000  # 日志级别
)

model.fit(train_pool, verbose=1000)  # 训练模型

# 预测测试集
test_preds = model.predict_proba(test_pool)[:, 1]  # 预测测试集的概率

In [ ]:
submission = test[['id']]
submission['Response'] = np.mean(preds, axis=0)

submission.to_csv('../submission/submission.csv', index=False)
submission